<a href="https://colab.research.google.com/github/username/detection-of-anthropogenic-river-avulsion/blob/main/notebooks/03_analysis_meijering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis: Ghost River Detection via NDRE & Meijering Filter

This notebook demonstrates the detection of the Iput River avulsion ('Ghost River') using Sentinel-2 Red Edge spectral indices and structural filtering.

**Methodology:**
1.  **Calculate NDRE**: Normalized Difference Red Edge index using Bands 05 and 08.
2.  **Vegetation Extraction**: Isolate the 'Green Strip' of dense vegetation that occupies the paleochannel.
3.  **Contrast Enhancement**: Improve the visibility of the features.
4.  **Structure Detection**: Apply Meijering/Frangi vesselness filters to detect the river-like morphology.

In [ ]:
import sys
import os
from pathlib import Path
import rasterio
import rasterio.enums
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure

# Add src to path
sys.path.append(os.path.abspath("../src"))

from filters import apply_structure_detection
from visualization import plot_structure_composite

## 1. Load Data

In [ ]:
# Load Custom Image (User Request - BEST RESULT)
custom_image_path = r"d:\pasha\RESEARCH\PALEO-HYDROLOGICAL_RECONSTRUCTION\results\visualizations\phase2_ndvi_ref_only_green.png"

print(f"Loading custom image: {custom_image_path}")
try:
    # Read image using matplotlib
    image = plt.imread(custom_image_path)
    
    if image.ndim == 3:
        # Assuming it's RGB/RGBA and the signal is in the Green channel (index 1)
        # The filename 'only_green' suggests this.
        ndre = image[:, :, 1] 
    else:
        ndre = image
        
    # Normalize to 0-1 if not already
    if np.max(ndre) > 1.0:
        ndre = ndre / 255.0
        
    # Note: We use the direct grayscale/green intensity as the input signal
    
    print(f"Image Loaded. Shape: {ndre.shape}, Range: {np.min(ndre):.2f} to {np.max(ndre):.2f}")

    plt.figure(figsize=(10, 6))
    plt.imshow(ndre, cmap='Greens')
    plt.colorbar(label='Input Signal (Green Channel)')
    plt.title("Custom Input Image")
    plt.show()

except Exception as e:
    print(f"Error loading image: {e}")
    raise e

# Ensure consistent data for filters
ndre = np.nan_to_num(ndre, nan=0.0)


## 2. Vegetation Extraction & Contrast Enhancement
We focus on the "Green Strip" - the paleo-channel is often characterized by denser vegetation (high NDRE) compared to fields.

In [ ]:
# Enhance contrast for the 'Green Strip'
# Standard linear rescaling for best results on this dataset
p2, p98 = np.percentile(ndre, (2, 98))
ndre_rescaled = exposure.rescale_intensity(ndre, in_range=(p2, p98))

plt.figure(figsize=(10, 6))
plt.imshow(ndre_rescaled, cmap='Greens')
plt.title("Enhanced Vegetation Contrast")
plt.axis('off')
plt.show()

## 3. Structural Detection (Meijering/Frangi)
We apply the vesselness filter to detect linear, continuous structures (paleo-channels) within the vegetation map.

### Parameters:
*   **sigmas**: Controls the scale (thickness) of detected features. Larger sigma values (e.g., 5-10) detect thicker rivers.
*   **method**: 'meijering'. Proven best for continuous ridges in this area.


In [ ]:
# Apply Filter to Enhanced NDRE
# Sigmas 2-12 for thicker features

bright_norm, dark_norm = apply_structure_detection(
    ndre_rescaled, 
    clip_max=1.0, 
    sigmas=range(2, 12, 2), 
    method='meijering'
) 

print("Filter Complete.")

## 4. Final Visualization
Composite of the original signal and the detected 'Veins' (channel).

**Tuning (User Approved):**
*   **Red / Bright Ridges**: Threshold lowered to **70th percentile** to highlight major features.
*   **Blue / Dark Ridges**: Threshold raised to **95th percentile** to reduce noise.

In [ ]:
# Visualize with tuned thresholds
output_path = "final_detection_result.png"
plot_structure_composite(ndre, bright_norm, dark_norm, output_path=output_path, percentiles=(70, 95))
print(f"Final result saved to {output_path}")